In [1]:
import numpy as np
import cv2

In [2]:
with open('coco.names.txt') as t:
    class_names=t.read().split('\n')

len(class_names)

81

The wights and cfg files were downloaded from: https://pjreddie.com/darknet/yolo/

In [3]:
# YOLOv3-320 was used in this notebook so slow performance for real-time might be noticed. 
# For faster performance use YOLOv3-tiny instead (might lead in decrease in accuracy)

model_config='yolov3.cfg.txt'

model_weights='yolov3.weights'

In [4]:
net=cv2.dnn.readNetFromDarknet(model_config,model_weights)

net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [6]:
def find_objects(outputs,img,threshold=0.5,nms_threshold=0.3):
    height,width,channels=img.shape
    bbox=[]
    classIds=[]
    confids=[]
    
    for output in outputs:
        for detection in output:
            scores=detection[5:]
            classId=np.argmax(scores)
            confidence=scores[classId]
            
            if confidence>threshold:
                w,h= int(detection[2]*width),int(detection[3]*height)
                x,y= int(detection[0]*width - w/2),int(detection[1]*height - h/2)
                
                bbox.append([x,y,w,h])
                classIds.append(classId)
                confids.append(float(confidence))
    
    indices=cv2.dnn.NMSBoxes(bbox,confids,threshold,nms_threshold)
     
    for i in indices:
        box=bbox[i]
        x,y,w,h=box
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,255),2)
        cv2.putText(img,f'{class_names[classIds[i]].upper()} {confids[i]*100:.1f}%', 
                    (x,y-10),cv2.FONT_HERSHEY_SIMPLEX, 0.4,(255,0,255),2)
        
       

In [39]:
#Testing on an image 

img=cv2.imread('')#Enter image path

blob=cv2.dnn.blobFromImage(img,1/255,(320,320),[0,0,0],crop=False)
net.setInput(blob)
    
outputs=net.getUnconnectedOutLayersNames()
outputs=net.forward(outputs)
    
find_objects(outputs,img)

cv2.imshow('Image',img)
cv2.namedWindow("Image", cv2.WINDOW_NORMAL)
cv2.waitKey(0)         
cv2.destroyAllWindows()

In [8]:
cap=cv2.VideoCapture(0)

while True:
    _,img=cap.read() 
    
    blob=cv2.dnn.blobFromImage(img,1/255,(320,320),[0,0,0],crop=False)
    net.setInput(blob)
    
    outputs=net.getUnconnectedOutLayersNames()
    outputs=net.forward(outputs)
    
    find_objects(outputs,img)
    
    cv2.imshow('Image',img)
    cv2.namedWindow("Image", cv2.WINDOW_NORMAL)
    
    if cv2.waitKey(1) == ord('a'):
        break
        
cap.release()    
cv2.destroyAllWindows()